In [19]:
import pandas as pd

from bs4 import BeautifulSoup
import requests

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import time
import calendar

## 1. Get press briefing links from https://www.whitehouse.gov/briefings-statements/

In [42]:
driver = webdriver.Chrome()

In [43]:
bf_list = []

In [45]:
for i in range(1,295):
    print(i)
    link = "https://www.whitehouse.gov/briefings-statements/page/" + str(i) + "/"
    driver.get(link)
    time.sleep(1)
    bfs = driver.find_elements_by_class_name("briefing-statement__content")
    for bf in bfs:
        bf_dict = {}

        try:
            s_type = bf.find_element_by_class_name("briefing-statement__type").text
        except:
            s_type = "not found"
        s_title = bf.find_element_by_class_name("briefing-statement__title").text
        s_url = bf.find_element_by_class_name("briefing-statement__title").find_element_by_tag_name("a").get_attribute('href')
        try:
            s_issue = bf.find_element_by_class_name("issue-flag").text
        except:
            s_issue = "not found"
        s_time = bf.find_element_by_tag_name("time").text

        bf_dict.update({
            "type" : s_type,
            "title" : s_title,
            "url" : s_url,
            "issue" : s_issue,
            "time" : s_time
        })

        bf_list.append(bf_dict)
    print("finished page", i)
    print("------------------------")


1
finished page 1
------------------------
2
finished page 2
------------------------
3
finished page 3
------------------------
4
finished page 4
------------------------
5
finished page 5
------------------------
6
finished page 6
------------------------
7
finished page 7
------------------------
8
finished page 8
------------------------
9
finished page 9
------------------------
10
finished page 10
------------------------
11
finished page 11
------------------------
12
finished page 12
------------------------
13
finished page 13
------------------------
14
finished page 14
------------------------
15
finished page 15
------------------------
16
finished page 16
------------------------
17
finished page 17
------------------------
18
finished page 18
------------------------
19
finished page 19
------------------------
20
finished page 20
------------------------
21
finished page 21
------------------------
22
finished page 22
------------------------
23
finished page 23
--------

finished page 180
------------------------
181
finished page 181
------------------------
182
finished page 182
------------------------
183
finished page 183
------------------------
184
finished page 184
------------------------
185
finished page 185
------------------------
186
finished page 186
------------------------
187
finished page 187
------------------------
188
finished page 188
------------------------
189
finished page 189
------------------------
190
finished page 190
------------------------
191
finished page 191
------------------------
192
finished page 192
------------------------
193
finished page 193
------------------------
194
finished page 194
------------------------
195
finished page 195
------------------------
196
finished page 196
------------------------
197
finished page 197
------------------------
198
finished page 198
------------------------
199
finished page 199
------------------------
200
finished page 200
------------------------
201
finished page

In [46]:
df_test = pd.DataFrame(bf_list)
df_test.head()

,issue,time,title,type,url
0,LAND & AGRICULTURE,"Jul 6, 2018",President Donald J. Trump Approves Texas Disas...,STATEMENTS & RELEASES,https://www.whitehouse.gov/briefings-statement...
1,IMMIGRATION,"Jul 6, 2018",Remarks by Vice President Pence at U.S. Immigr...,REMARKS,https://www.whitehouse.gov/briefings-statement...
2,ECONOMY & JOBS,"Jul 6, 2018","Jobs, Jobs, Jobs: The American Economy Is Booming",NEWS CLIPS,https://www.whitehouse.gov/briefings-statement...
3,INFRASTRUCTURE & TECHNOLOGY,"Jul 6, 2018",Presidential Message on National Air Traffic C...,STATEMENTS & RELEASES,https://www.whitehouse.gov/briefings-statement...
4,FOREIGN POLICY,"Jul 5, 2018",Readout of the Vice President’s Call with Mace...,STATEMENTS & RELEASES,https://www.whitehouse.gov/briefings-statement...


## Get day of the week from `time`

In [69]:
def get_weekday(row):
    day = datetime.datetime.strptime(row["time"], "%b %d, %Y")
    weekday = calendar.day_name[day.weekday()]
    return weekday

In [70]:
df_test["weekday"] = df_test.apply(get_weekday, axis=1)

In [72]:
df = df_test

In [74]:
df.weekday.value_counts()

Friday       613
Tuesday      610
Thursday     580
Wednesday    564
Monday       505
Saturday     168
Sunday       130
Name: weekday, dtype: int64

In [82]:
df.groupby("type").weekday.value_counts(normalize = True, )*100

type                   weekday  
FACT SHEETS            Tuesday      26.339286
                       Thursday     21.875000
                       Wednesday    20.089286
                       Monday       13.392857
                       Friday       12.946429
                       Sunday        3.571429
                       Saturday      1.785714
NEWS CLIPS             Tuesday      22.335025
                       Wednesday    19.796954
                       Friday       19.289340
                       Thursday     19.289340
                       Monday       16.243655
                       Saturday      1.522843
                       Sunday        1.522843
PRESS BRIEFINGS        Thursday     22.794118
                       Tuesday      22.058824
                       Monday       20.955882
                       Wednesday    16.176471
                       Friday       15.808824
                       Saturday      1.470588
                       Sunday        0.735294
R

In [87]:
df.shape

(3170, 6)

In [86]:
df[df.type == "STATEMENTS & RELEASES"].shape

(1677, 6)

In [88]:
df.to_csv("wh_briefings.csv", index = False)

In [16]:
df1 = pd.read_csv("wh_briefings.csv")
df1.shape

(3170, 6)

In [17]:
df1[df1["type"] == "STATEMENTS & RELEASES"].weekday.value_counts()

Friday       375
Wednesday    292
Tuesday      279
Thursday     266
Monday       252
Saturday     116
Sunday        97
Name: weekday, dtype: int64

## Get text for each url

In [44]:
def get_content(row):
    
    url = row["url"]
    raw_html = requests.get(url).content
    soup = BeautifulSoup(raw_html, "html.parser")

    content = soup.find("div", attrs={"class": "page-content"}).find_all("p")
    content_list = []
    for c in content:
        sentence = c.string
        content_list.append(sentence)
    
    return content_list

In [46]:
df1["content"] = df1.apply(get_content, axis=1)

In [48]:
df1.to_csv("trump_w_content", index = False)